In [35]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/aqf/data/kolkata.csv', parse_dates=True)
df.set_index('date')
df.head()

,date,PM25
0,2015-01-01,230.0
1,2015-01-02,143.0
2,2015-01-03,119.0
3,2015-01-04,160.0
4,2015-01-05,154.0


In [36]:
from utility import sequence_to_table
from numpy import mean

seq_mean = mean(df.PM25.values)

sequence = df.PM25.values - seq_mean
data = sequence_to_table(sequence, look_back=30)

# divide data into feature and target
X = data.drop(columns=['next']).values
y = data.next.values

# first reshape the data
X_reshaped = X.reshape((X.shape[0], 1, X.shape[1]))

Implementing RNN-SC
-----------

In [37]:
# 1-layer RNN model

import tensorflow as tf
from keras.models import Sequential
tf.random.set_seed(42)

model = Sequential()

model.add(tf.keras.layers.SimpleRNN(units=64, 
                              activation='relu', 
                              input_shape=(1, X_reshaped.shape[2])))
model.add(tf.keras.layers.Dense(1))

model.compile(loss='mse', optimizer='adam')

In [ ]:
import tensorflow as tf
from keras.models import Sequential
tf.random.set_seed(42)

model = Sequential()
model.add(tf.keras.layers.SimpleRNN(units=512, 
                              activation='relu', 
                              input_shape=(1, X_reshaped.shape[2]), return_sequences=True))
model.add(tf.keras.layers.SimpleRNN(units=512, 
                              activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.compile(loss='mse', optimizer='adam')

In [38]:
history = model.fit(X_reshaped, y, epochs=100, batch_size=32)

Epoch 1/100
79/79 [==============================] - 1s 2ms/step - loss: 2007.5996
Epoch 2/100
79/79 [==============================] - 0s 2ms/step - loss: 1040.5166
Epoch 3/100
79/79 [==============================] - 0s 2ms/step - loss: 844.1450
Epoch 4/100
79/79 [==============================] - 0s 3ms/step - loss: 754.4965
Epoch 5/100
79/79 [==============================] - 0s 2ms/step - loss: 695.5621
Epoch 6/100
79/79 [==============================] - 0s 2ms/step - loss: 660.5488
Epoch 7/100
79/79 [==============================] - 0s 2ms/step - loss: 649.2213
Epoch 8/100
79/79 [==============================] - 0s 2ms/step - loss: 626.1021
Epoch 9/100
79/79 [==============================] - 0s 2ms/step - loss: 631.0075
Epoch 10/100
79/79 [==============================] - 0s 2ms/step - loss: 608.5111
Epoch 11/100
79/79 [==============================] - 0s 2ms/step - loss: 596.5701
Epoch 12/100
79/79 [==============================] - 0s 2ms/step - loss: 593.2334
Epoch 13/10

In [39]:
forecast = model.predict(X_reshaped)

In [40]:
from sklearn.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(y, forecast))

21.352433237684615

In [41]:
import numpy as np

predictions = []
input = X_reshaped[-1]

for i in range(15):
  input = input.reshape(1, 1, 30)
  predict = model.predict(input)
  input = input.ravel()
  predictions.append(round(predict[0][0] + seq_mean, 2))
  input = np.delete(input, 0)
  input = np.append(input, predict[0][0])

In [42]:
for i in predictions:
  print(i)

218.88
223.43
217.65
215.32
178.58
183.9
181.56
176.58
180.57
202.57
214.69
210.8
229.51
227.98
227.28
